In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import model_selection, naive_bayes, svm

In [3]:
data=[]
data_text=[]
with open('dataset.json', 'r', encoding='utf-8') as ene_file:
    cnt=0;
    for line in ene_file.readlines():
        line = str(line).strip()
        # #workaround for escape characters
        # if "\'" in line:
        #     line = line.replace("\'", "\\'")
        data.append(json.loads(line))
        cnt+=1
        if cnt<10:
            pass
            #print(line)

In [4]:
df = pd.DataFrame(columns=['page_id','text','category'])

In [5]:
for i in range(0,30492):
    df.loc[i]=[int(data[i].get('page_id')),data[i].get('text'),int(data[i].get('classes')[0])]
    data_text.append(data[i].get('text'))

In [6]:
df.astype({'category': 'int64'},copy=False).dtypes

page_id     object
text        object
category     int64
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30492 entries, 0 to 30491
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   page_id   30492 non-null  object
 1   text      30492 non-null  object
 2   category  30492 non-null  object
dtypes: object(3)
memory usage: 952.9+ KB


In [8]:
X=df['text'].values
y=df['category'].values

In [9]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X, y,test_size=0.1,stratify=y)

In [10]:
Encoder = LabelEncoder()
train_y = Encoder.fit_transform(train_y)
test_y = Encoder.fit_transform(test_y)

In [11]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data_text)
train_x_Tfidf = Tfidf_vect.transform(train_x)
test_x_Tfidf = Tfidf_vect.transform(test_x)

In [12]:
#print(Tfidf_vect.vocabulary_)
Naive = naive_bayes.MultinomialNB()
Naive.fit(train_x_Tfidf,train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(train_x_Tfidf,train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
predictions_NB = Naive.predict(test_x_Tfidf)
prediction_svm=SVM.predict(test_x_Tfidf)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions_NB)

0.7852459016393443

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, prediction_svm)

0.8563934426229508

In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predictions_NB))

              precision    recall  f1-score   support

           0       0.65      0.30      0.41       563
           1       0.78      0.96      0.86      2108
           2       0.99      0.69      0.81       280
           3       0.84      0.53      0.65        30
           4       1.00      0.01      0.03        69

    accuracy                           0.79      3050
   macro avg       0.85      0.50      0.55      3050
weighted avg       0.78      0.79      0.75      3050



In [18]:
from sklearn.metrics import classification_report
print(classification_report(test_y, prediction_svm))

              precision    recall  f1-score   support

           0       0.72      0.60      0.65       563
           1       0.87      0.93      0.90      2108
           2       0.98      0.95      0.96       280
           3       0.79      1.00      0.88        30
           4       0.94      0.22      0.35        69

    accuracy                           0.86      3050
   macro avg       0.86      0.74      0.75      3050
weighted avg       0.85      0.86      0.85      3050

